# Alpha Test

Test ideas relating to alpha racing

Created by Michael George (AKA Logiqx)

Website: https://logiqx.github.io/gps-wizard/

In [1]:
import os

from common_core import Printable, projdir, loadCsvIntoArray

## Run Unit Tests

In [2]:
if __name__ == '__main__':
    x = loadCsvIntoArray(os.path.join(projdir, 'sessions', '20220404', '20220404-apex-pro.csv'))
    
    print(x)

[(1018010516, 620964878, -6787940, 6.00000e+00, 1.283,    26, 14, 0.8,  28.)
 (1018010517, 620964997, -6787940, 7.50000e+00, 1.283,    27, 14, 0.8,  44.)
 (1018010518, 620965108, -6787901, 9.00000e+00, 1.181,    28, 14, 0.8,  99.)
 ...
 (1018022326, 620963071, -6783868, 4.05970e+04, 1.386, 11836, 12, 0.7,  92.)
 (1018022327, 620963054, -6783658, 4.05985e+04, 1.181, 11837, 12, 0.7, 102.)
 (1018022328, 620962986, -6782826, 4.06000e+04, 1.078, 11838,  0, 0.7, 105.)]
